In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
import torch.nn as nn
import pytorch_lightning as pl
import yfinance as yf
import matplotlib.pyplot as plt
from collections import namedtuple
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os

#### Model

In [ ]:
class LSTM_Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        input_dim, hidden_dim, dropout = config["input_dim"], config["hidden_dim"], config["dropout"] 
        num_layers, output_dim = config["num_layers"], config["output_dim"]
        seq_len = config["forecast_len"]
        training_len = config["training_len"]

        self.forecast_len = seq_len

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # input shape: [batch * seq_len * d_model]
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)

        #output layer
        self.linear = nn.Linear(hidden_dim * training_len, seq_len)

        self._device = config["device"]
        self.T = config["bptt"]

        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, x):
        # initial hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(self._device)
        #initialise cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(self._device)
        
        out, (hi, ci) = self.lstm(x, (h0.detach(), c0.detach())) 
        
        out = self.linear(out.contiguous().view(len(x), -1))
 
        return out


In [ ]:
device = ("cuda:0" if torch.cuda.is_available else "cpu")

params = dict(
    input_dim = 1,
    hidden_dim = 64,
    training_len = 60,
    forecast_len = 30, 
    val_len = 60,
    dropout = 0.1,
    num_layers = 2,
    output_dim = 1,
    lr= 1e-3,
    device=device,
)

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def get_prediction(t):
    return [arr.view(-1) for arr in t]


In [ ]:
train_output_img_dir = f"train_output_imgs/lstm/^GSPC/"


In [ ]:
class LSTMTrainer(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters()

        self.model = LSTM_Model(config)
        self.lr = config["lr"]
        self.training_len = config["training_len"]
        self.forecast_len = config["forecast_len"]

        self.criterion = nn.MSELoss(reduction='mean')

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def calc_pearson_coeff(self, y_pred, y):

        predicted = y_pred
        targets = y
        vy_pred = predicted - torch.mean(predicted)
        vy = targets - torch.mean(targets)
        denom = torch.sum(vy_pred ** 2) * torch.sum(vy ** 2)

        corr = torch.sum(vy_pred * vy) / torch.sqrt(denom)

        return corr

    def training_step(self, batch, batch_idx):
        src, targets = batch
        all_preds = [] 

        all_preds = self.model(src)
            
        y_hat = all_preds
        y = targets.squeeze(-1) 

        loss = self.criterion(y_hat, y)

        pearson_coeff = self.calc_pearson_coeff(y_hat, y)

        self.log_dict({"train_loss":loss, "train_pearson_coeff":pearson_coeff}, prog_bar=True, on_epoch=True, on_step=True, logger=True)

        return {"loss": loss}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack(
            [x["loss"].detach() for x in outputs]).mean()
        
        self.log_dict({"train_loss_epoch": avg_loss, "step":self.current_epoch})

        if self.current_epoch % 5 == 0:
            self.print_pred(train_output_img_dir+"train")

    def validation_step(self, batch, batch_idx):

        src, targets = batch
        all_preds = []

        all_preds = self.model(src)
            
        y_hat = all_preds
        y = targets.squeeze(-1)

        loss = self.criterion(y, y_hat)

        pearson_coeff = self.calc_pearson_coeff(y_hat, y)

        self.log_dict({"val_loss":loss, "train_pearson_coeff":pearson_coeff}, prog_bar=True, on_epoch=True, on_step=True, logger=True)

        return {"loss": loss}
    
    def print_pred(self, file_prefix):
        with torch.no_grad():
            all_preds = []
            init_train_data = []
            all_targets = []

            for step, trng_data in enumerate(train_loader_unshuffled):
                if step == 2:
                    break

                src, targets = trng_data
            
                all_targets.append(targets.detach().cpu())

                if step == 0:
                    init_train_data = src[0, :, 0].reshape(-1)
                
                
                preds = self(trng_data)
                all_preds.append(preds.detach().cpu())


            all_preds = np.array(flatten(get_prediction(all_preds)))
            all_targets = np.array(flatten(get_prediction(all_targets)))

            #inverse transform

            init_train_data = features_scaler.inverse_transform(init_train_data.reshape(-1, 1)).reshape(-1)
            all_preds = features_scaler.inverse_transform(all_preds.reshape(-1, 1)).reshape(-1)
            all_targets = features_scaler.inverse_transform(all_targets.reshape(-1, 1)).reshape(-1)

            end_plot_idx = self.training_len + len(all_preds)
            plt.figure(figsize=(8, 6))
            plt.plot(init_train_data, label='trailing')
            plt.plot(np.arange(self.training_len, end_plot_idx), all_preds, label="predicted")
            plt.plot(np.arange(self.training_len, end_plot_idx), all_targets, label="actual")
            plt.title(f"Validation prediction for epoch {self.current_epoch}")
            plt.legend()
            plt.grid()

            if not os.path.exists(file_prefix):
                os.makedirs(file_prefix)

            plt.savefig(f"{file_prefix}/First_128_preds_Epoch_{self.current_epoch}.jpg", bbox_inches="tight")
            
            plt.close()

             

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack(
            [x['loss'].detach() for x in outputs]).mean()

        self.log("ptl/val_loss", avg_loss)
        
        self.log_dict({"valid_loss_epoch": avg_loss, "step":self.current_epoch})

        # if self.current_epoch % 5 == 0:
        #     self.print_pred(train_output_img_dir+"val")
    

    def forward(self, x):
        src, targets = x
        all_preds = []

        src = src.cuda()

        all_preds = self.model(src)
            
        return all_preds

        


#### Creating Datasets

In [ ]:
FeatureTargetSet = namedtuple('FeatureTargetSet', ['train_features', 'target_values'])

In [ ]:
class FeatureDataset(Dataset):
    def __init__(self, features, targets, training_len, forecast_len):
        super().__init__()
        self.features = features
        self.targets = targets
        self.training_len = training_len
        self.forecast_len = forecast_len
        self.feature_len = len(features)

    def __len__(self):
        return self.feature_len - self.training_len - self.forecast_len + 1
    
    def __getitem__(self, idx):
        end_trng_idx = idx + self.training_len
        end_target_idx = end_trng_idx + self.forecast_len

        train_features = torch.as_tensor(self.features[idx:end_trng_idx], dtype=torch.float32)   
        targets = torch.as_tensor(self.targets[end_trng_idx:end_target_idx], dtype=torch.float32)  
        return FeatureTargetSet(train_features=train_features, target_values=targets)

#### Get prices data

In [ ]:
def download_data(codes, start_date, end_date):
    data = yf.download(codes, start_date, end_date)

    if len(codes) == 1:
        data.columns = [data.columns, codes*len(data.columns)]

    return data.dropna()

In [ ]:
tickers = ["^GSPC", "AAPL", "MSFT", "NKE", "JPM", "JNJ", "BTC-USD"]

In [ ]:
daily_data = download_data(tickers, "2018-05-01", "2022-05-01")
close_data = daily_data["Adj Close"]

In [ ]:
prices_data = close_data['MSFT'].values
prices_data = prices_data.reshape(-1)

plt.plot(prices_data)

#### Splitting Data

In [ ]:
ticker = "MSFT"
features = close_data[ticker].values

In [ ]:
def split_data(features):
    train_idx = int(features.shape[0]*0.6)
    val_idx = int(features.shape[0] * 0.2)
    train_features = features[:train_idx]
    val_features = features[train_idx:train_idx + val_idx]
    test_features = features[train_idx + val_idx:]

    return train_features, val_features, test_features

In [ ]:
train_features, val_features, test_features = split_data(features)

In [ ]:
train_features[:5]

#### Preprocessing Data

In [ ]:
features_scaler = MinMaxScaler()
features_scaler.fit(train_features.reshape(-1, 1))

In [ ]:
def preprocess_data(train_features, val_features, test_features):

    features_scaler.fit(train_features.reshape(-1, 1))
    
    f_train_values = features_scaler.transform(train_features.reshape(-1, 1))
    f_val_values = features_scaler.transform(val_features.reshape(-1, 1))
    f_test_values = features_scaler.transform(test_features.reshape(-1, 1))

    return f_train_values, f_val_values, f_test_values



In [ ]:
f_train_values, f_val_values, f_test_values = preprocess_data(train_features, val_features, test_features)

#### Parameters

In [ ]:
device = ("cuda:0" if torch.cuda.is_available else "cpu")

In [ ]:
params = dict(
    input_dim = 1,
    hidden_dim = 64,
    training_len = 60,
    forecast_len = 30,
    val_len =60,
    dropout = 0.1,
    num_layers = 2
    output_dim = 1,
    lr= 1e-3,
    device=device,
    bptt = 10,
)

#### DataLoaders

In [ ]:
training_len = params['training_len']
forecast_len = params['forecast_len']

In [ ]:
batch_size = 64

train_dataset = FeatureDataset(f_train_values, f_train_values, params['training_len'], forecast_len)
val_dataset = FeatureDataset(f_val_values, f_val_values, params['val_len'], forecast_len)
test_dataset = FeatureDataset(f_test_values, f_test_values, params['val_len'], forecast_len)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=8)
train_loader_unshuffled = DataLoader(train_dataset, batch_size, shuffle=False, num_workers=8)


In [ ]:
features, target = next(iter(train_loader))
print(features.shape)
print(target.shape)

#### Training model

In [ ]:
log_dir = "lstm_logs"
model_dir = "lstm_models"

device = ("cuda:0" if torch.cuda.is_available else "cpu")

def train(ticker, version_name = "", model_name="", ckpt_dir = "w2v"):

    if version_name == "":
        version_name = ticker
    
    if model_name == "":
        model_name = ticker

    logger = TensorBoardLogger(
        save_dir=log_dir,
        ### TODO: change version when reruun
        version=f'{version_name}_{ckpt_dir}'
    )

    checkpoint_callback = ModelCheckpoint(
        monitor="valid_loss_epoch",
        mode="min",
        ### TODO: change dir path when rerun
        dirpath=f"{model_dir}/{ckpt_dir}/{ticker}",
        filename="{epoch}-{valid_loss_epoch:.4f}",
        save_last= True,
        save_top_k=2
    )

    early_stopping_callback = EarlyStopping(
        monitor="valid_loss_epoch",
        mode="min",
        patience=30
    )


    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1,
        logger=logger,
        callbacks=[checkpoint_callback, early_stopping_callback],
        log_every_n_steps=9,
        # deterministic=True
    )

    stock_model = LSTMTrainer(params)

    trainer.fit(stock_model, train_loader, val_loader)

    return trainer, stock_model

#### predictions

In [ ]:
trainer, stock_model = train(ticker)

#### Evaluation metrics

In [ ]:
def calculate_rmse(y_true, y_pred):

    rmse = np.sqrt(np.mean((y_true-y_pred)**2))                   
    return rmse

def calculate_mape(y_true, y_pred): 

    y_pred = np.array(y_pred)
    y_true = np.array(y_true)    
    mape = np.mean(np.abs((y_true-y_pred) / y_true))*100    
    return mape


def calc_pearson_coeff(y_pred, y):
    predicted = y_pred
    targets = y
    vy_pred = predicted - np.mean(predicted)
    vy = targets - np.mean(targets)
    denom = np.sum(vy_pred ** 2) * np.sum(vy ** 2)

    corr = np.sum(vy_pred * vy) / np.sqrt(denom)

    return corr

def mse_loss(pred, target, reduction='sum'):
    loss = np.sum(np.square(pred - target))
    if reduction == "sum":
        return loss
    else:
        return loss / len(pred)

In [ ]:
def get_prediction_value(preds):
    return [x.cpu().detach().numpy() for x in preds]

def concat_preds(preds):
    return np.concatenate(preds, axis=0)

def revert_transform(values):
    return features_scaler.inverse_transform(values.reshape(-1, 1)).reshape(-1)

def forecast(stock_model, trainer, ckpt_path, test_loader):
    with torch.no_grad():
        preds = trainer.predict(dataloaders=test_loader, model=stock_model, ckpt_path=ckpt_path)

    all_preds = concat_preds(get_prediction_value(preds))

    return all_preds

def evaluate(pred, y):
    pred = revert_transform(pred)
    y = revert_transform(y)
    
    rmse = calculate_rmse(y, pred)
    mse = rmse ** 2
    mape = calculate_mape(y, pred)

    r = calc_pearson_coeff(pred, y)

    res = dict(
        rmse = rmse,
        mse =mse,
        mape =mape,
        r = r
    )

    return res

In [ ]:
ckpt_path = 'lstm_models/MSFT/epoch=11-valid_loss_epoch=0.0017.ckpt'
all_preds = forecast(stock_model, trainer, ckpt_path, test_loader)
eval_res = evaluate(all_preds, f_test_values[training_len:])


In [ ]:
savefig = False
img_folder =f"output_imgs/lstm/{ticker}/"

def gen_fig(preds, eval_res, targets, img_folder, title, savefig=False):
    with torch.no_grad():
        if not os.path.exists(img_folder):
            os.makedirs(img_folder)
        
        start_idx = training_len 
        end_idx = training_len + forecast_len 
        mse = eval_res["mse"]

        filename = f"{img_folder}{ticker}_results.txt" 
        if savefig:
            if os.path.exists(filename):
                f = open(filename, "a")
                f.write("\n")
            else:
                f = open(filename, "x")

        for metric in eval_res.keys():
            print(f"Metric {metric}: {eval_res[metric]}")
            if savefig:
                f.write(f"Metric {metric}: {eval_res[metric]}\n")

        plt.figure(figsize=(8, 6))
        range_start_idx = training_len
        range_end_idx = training_len + len(preds)
        # predictions_flattened = np.array(all_preds)
        plt.plot(list(range(range_start_idx, range_end_idx)), revert_transform(preds), label="predicted")
        plt.plot(list(range(range_start_idx, range_end_idx)), revert_transform(targets), label="target")
        plt.plot(revert_transform(f_test_values[:start_idx]), label="trailing")
        plt.title(f"{title}")
        plt.legend()
        plt.grid()

        if savefig:
            plt.savefig(f"{img_folder}_all_preds.jpg", bbox_inches="tight")

        plt.show()

#### Train models on all stock data

In [ ]:
tickers = ["^GSPC", "AAPL", "MSFT", "NKE", "JPM", "JNJ"]

daily_data = download_data(tickers, "2018-05-02", "2022-05-01")
prices_data = daily_data["Adj Close"]

In [ ]:
ticker_test = ["^GSPC", "AAPL", "NKE", "JPM", "JNJ"]

ckpt_dir = "lstm_n_step"

for ticker in ticker_test:
        # print(prices_data)
        features = prices_data[ticker].values

        train_features, val_features, test_features = split_data(features)
        f_train_values, f_val_values, f_test_values = preprocess_data(train_features, val_features, test_features)
        
        training_len = 60
        forecast_len = 30

        batch_size = 64

        train_dataset = FeatureDataset(f_train_values, f_train_values, params['training_len'], forecast_len)
        val_dataset = FeatureDataset(f_val_values, f_val_values, params['val_len'], forecast_len)
        test_dataset = FeatureDataset(f_test_values, f_test_values, params['val_len'], forecast_len)

        train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=8)
        val_loader = DataLoader(val_dataset, batch_size, shuffle=False, num_workers=8)
        test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=8)
        train_loader_unshuffled = DataLoader(train_dataset, batch_size, shuffle=False, num_workers=8)
        
        src, target = next(iter(train_loader))
        print(src.shape)
        print(target.shape)
        
        global train_output_img_dir
        train_output_img_dir = f"train_output_imgs/{ckpt_dir}/{ticker}/"
        
        trainer, stock_model = train(ticker, version_name=f"{ticker}", model_name=f"{ticker}", ckpt_dir=ckpt_dir)

In [ ]:
tickers = ["^GSPC", "AAPL", "MSFT", "NKE", "JPM", "JNJ"]

daily_data = download_data(tickers, "2018-05-02", "2022-05-01")
prices_data = daily_data["Adj Close"]

In [ ]:
ticker = "AAPL"

features = prices_data[ticker].values

train_features, val_features, test_features = split_data(features)
f_train_values, f_val_values, f_test_values = preprocess_data(train_features, val_features, test_features)
test_dataset = FeatureDataset(f_test_values, f_test_values, params['val_len'], forecast_len)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=8)

In [ ]:
# title = f"1-step ahead price prediction for {ticker}"

savefig = True
stock_model = LSTMTrainer(params)
ckpt_path = 'lstm_models/lstm_n_step/AAPL/epoch=5-valid_loss_epoch=0.0230.ckpt'
all_preds = forecast(stock_model, trainer, ckpt_path, test_loader)
eval_res = evaluate(all_preds[0], f_test_values[training_len:training_len + forecast_len])
targets = f_test_values[training_len:training_len + forecast_len]
img_folder = f"output_imgs/{ckpt_dir}/{ticker}/"
fig_title = f"30 step ahead prediction with prices for {ticker}"
gen_fig(all_preds[0], eval_res, targets, img_folder, fig_title, savefig=savefig)